In [1]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
import cv2
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
from glob import glob
from PIL import Image
import random
import albumentations as A
import keras
import matplotlib.cm as cm
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, recall_score, accuracy_score, precision_score, f1_score


In [2]:
files = ['Normal', 'COVID', 'Lung_Opacity','Viral Pneumonia']
path = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset"
data_dir = os.path.join(path)

data = []
for id, level in enumerate(files):
    for file in os.listdir(os.path.join(data_dir, level+'/'+'images')):
#         data.append(['{}/{}'.format(level, file), level])
        data.append([level +'/' +'images'+ '/'+file, level])


data = pd.DataFrame(data, columns = ['image_file', 'label'])

data['path'] = path + '/' +data['image_file']
data['label'] = data['label'].map({'Normal': 'Normal', 'COVID': 'Covid_positive', 'Lung_Opacity':'Lung_Opacity', 'Viral Pneumonia':'Viral_Pneumonia'})

data.head()


,image_file,label,path
0,Normal/images/Normal-859.png,Normal,../input/covid19-radiography-database/COVID-19...
1,Normal/images/Normal-158.png,Normal,../input/covid19-radiography-database/COVID-19...
2,Normal/images/Normal-10121.png,Normal,../input/covid19-radiography-database/COVID-19...
3,Normal/images/Normal-1811.png,Normal,../input/covid19-radiography-database/COVID-19...
4,Normal/images/Normal-97.png,Normal,../input/covid19-radiography-database/COVID-19...


In [3]:
data['image'] = data['path'].map(lambda x: np.asarray(Image.open(x).resize((100,100))))

In [4]:
all_data = []

# Storing images and their labels into a list for further Train Test split
{'Normal': 'Normal', 'COVID': 'Covid_positive', 'Lung_Opacity':'Lung_Opacity', 'Viral Pneumonia':'Viral_Pneumonia'}
for i in range(len(data)):
    image = cv2.imread(data['path'][i])  
    image = cv2.resize(image, (100, 100)) / 255.0
    label = 0
    if data['label'][i] == "Covid_positive":
        label = 0
    else:
        label = 1
    all_data.append([image, label])


In [6]:
x = []
y = []

for image, label in all_data:
    x.append(image)
    y.append(label)

# Converting to Numpy Array
x = np.array(x)
y = np.array(y)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_val, y_val, test_size=0.5, random_state=42)


print(x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape)

(16932, 100, 100, 3) (2117, 100, 100, 3) (2116, 100, 100, 3) (16932,) (2117,) (2116,)


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, DenseNet121
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
input_size = (100, 100, 3)

model1 = tf.keras.models.load_model('/kaggle/input/mymodel-4-0/model.h5')
model1.trainable = False 

# Example: Load a pre-trained EfficientNet model
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=Input(shape=input_size))
efficientnet_model.trainable = False

# Example: Load a pre-trained DenseNet model
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=input_size))
densenet_model.trainable = False

inputs = Input(shape=input_size)
output_model1 = model1(inputs)
output_efficientnet = efficientnet_model(inputs)
output_densenet = densenet_model(inputs)

output_model1 = Flatten()(output_model1)
output_efficientnet = Flatten()(output_efficientnet)
output_densenet = Flatten()(output_densenet)

combined = Concatenate()([output_model1, output_efficientnet, output_densenet])

outputs = Dense(2)(combined)


com_model = Model(inputs, outputs)

com_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

history = com_model.fit(x_train, y_train, epochs=10,
                        validation_data=(x_test, y_test))

Epoch 1/10
 14/530 [..............................] - ETA: 6:45 - loss: 2.4317 - accuracy: 0.7411

In [ ]:

com_model.save('combined_model.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc = 'lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc = 'upper right')
plt.show()

In [ ]:
pd.DataFrame(history.history).plot()
plt.title('Model Accuracy/Loss')
plt.ylabel('Accuracy/Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Make predictions on the test set
y_pred_one_hot = com_model.predict(x_val)
y_pred = np.argmax(y_pred_one_hot, axis=1)  # Convert one-hot predictions to class labels

# Compute evaluation metrics
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(2), labels=['Covid_negative','Covid_positive'])
plt.yticks(np.arange(2), labels=['Covid_negative','Covid_positive'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, str(conf_matrix[i, j]), ha='center', va='center', color='red' if conf_matrix[i, j] < conf_matrix.max() / 2 else 'black')
plt.show()


# Compute the confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)

# Compute the percentage confusion matrix
conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Print the percentage confusion matrix with class labels
print("Percentage Confusion Matrix:")
print(conf_matrix_percentage)

# Plot the percentage confusion matrix with class labels
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix_percentage, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(2), labels=['Covid_negative','Covid_positive'])
plt.yticks(np.arange(2), labels=['Covid_negative','Covid_positive'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, f'{conf_matrix_percentage[i, j]*100:.2f}%', ha='center', va='center', color='red' if conf_matrix_percentage[i, j] < 0.5 else 'black')
plt.show()